notebook/requirements.txt: EDA를 위한 종속성 관리
autoEDA: llama2로 특수 문자 여부를 판단한 내용

In [13]:
import pandas as pd
import pyarrow as pa
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [4]:
file_path = '../data/train_dataset/train/dataset.arrow'
with pa.memory_map(file_path, 'r') as source:
    table = pa.ipc.open_stream(source).read_all()
df: pd.DataFrame = table.to_pandas()
df.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548


In [6]:
text_column = 'context'  # 예: 'context', 'document' 등
if text_column not in df.columns:
    print(f"Warning: '{text_column}' column not found. Please specify the correct column name.")

\* 로컬 Ollama 서버로 llama2 활용하는 방법 (macOS 기준)
1. Ollama 설치
    ```
    brew install ollama
    ```
2. Ollama 서버 실행
    ```
    ollama run llama2
    ```
3. Ollama 서버 중단
    ```
    pkill ollama
    ```

In [7]:
llm = OllamaLLM(model="llama2")

In [14]:
text_analysis_template = """
다음 텍스트를 분석하고 다음 질문에 답하세요:

텍스트: {text}

1. 이 텍스트에 이상한 점이 있나요? 있다면 무엇인가요?
2. 이 텍스트에 있는 특수문자들이 의미적으로 필요한가요? 필요하지 않은 특수문자가 있다면 어떤 것인가요?
3. 이 텍스트에 전처리가 필요한 부분이 있나요? 있다면 어떤 전처리를 제안하시겠습니까?
4. 이 텍스트의 주요 주제나 키워드는 무엇인가요?

답변을 간결하게 작성해 주세요.
"""

text_analysis_prompt = PromptTemplate(
    input_variables=["text"],
    template=text_analysis_template
)

text_analysis_chain = (
    {"text": RunnablePassthrough()} 
    | text_analysis_prompt 
    | llm 
    | StrOutputParser()
)

In [9]:
def analyze_special_chars(text):
    # 한글, 영문, 숫자, 일반적인 문장 부호를 제외한 문자를 특수문자로 간주
    special_chars = re.findall(r'[^\w\s\.\,\!\?\"\'\:\;\-\(\)]', text)
    return list(set(special_chars))

In [16]:
sample_size = min(100, len(df))  # 전체 데이터셋이나 최대 100개의 샘플 분석
sample_indices = np.random.choice(len(df), sample_size, replace=False)

for idx in sample_indices:
    text = df.iloc[idx][text_column]  # 'text' 컬럼명을 실제 데이터에 맞게 변경하세요
    special_chars = analyze_special_chars(text)
    
    print(f"Sample {idx}:")
    print("Text:", text[:100] + "..." if len(text) > 100 else text)
    print("Special characters:", special_chars)
    
    result = text_analysis_chain.invoke(text)
    print("LLM Analysis:")
    print(result)
    print("-" * 50)

Sample 686:
Text: 하후연의 요청으로 조조는 군사를 이끌고 한중으로 향했으나 내부의 우환으로 인해 장안에서 잠시 군사를 멈춰 상황을 지켜보기만 하였다. 219년 정월, 유비가 밤에 녹각을 불사르며 포...
Special characters: ['\\']
LLM Analysis:
 answers:

1. Yes, there are some unusual characters in the text. The character "炽" (fēi) is used to indicate the sound of a flame, while the character "擊" (jiào) is used to indicate a violent or fierce attack. These characters are not commonly used in everyday Chinese texts and may be specific to this historical context.
2. Yes, there are some special characters in the text that have specific meanings. The character "炽" (fēi) is used to indicate the sound of a flame, while the character "擊" (jiào) is used to indicate a violent or fierce attack.
3. Yes, there are some parts of the text that could benefit from preprocessing or filtering. For example, there are several instances of repeated characters in the text, such as "하" (hā) and "조" (zhōu), which could be filtered out to improve readability. Additionally, there are some c